In [1]:
import pandas as pd

In [42]:
df = pd.read_csv('data/list.csv')
df.head()

df["volume"] = df["x"] * df["y"] * df["z"]

#min and max
print(df['volume'].min()/1000)
print(df['x'].min())
print(df['y'].min())
print(df['z'].mean())
print(df['Waga (kg)'].max())

6000.0
300
200
233.07692307692307
40


In [90]:
class Cuboid:
    width = 0
    height = 0
    depth = 0

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth


class Package(Cuboid):
    x = 0
    y = 0
    z = 0

    def __init__(self, dimensions, location, depth):
        width, height, depth = dimensions
        super().__init__(self, width, height, depth)

    def addLocation(self, location):
        self.x, self.y, self.z = location


class Pallet(Cuboid):
    packages = []

    def __init__(self):
        super().__init__(800, 1200, 2000 - 144)

    def addPackage(self, package, location):
        package.addLocation(location)
        self.packages.append(package)

def loadPackageTypes():
    packageTypes = pd.read_csv("data/listaProduktow.csv")
    return packageTypes

def loadOrder(orderNo):
    order = pd.read_csv(f"data/Zamowienia/Zamowienie{orderNo}.csv")
    return order

In [43]:
def loadPackageTypes():
    packageTypes = pd.read_csv("data/listaProduktow.csv")
    packageTypes.set_index("ID Produktu", inplace=True)
    packageTypes[['x', 'y','z']] = packageTypes['Wymiary (mm)'].str.split('x', n=2, expand=True)
    packageTypes.drop(columns=['Wymiary (mm)'], inplace=True)


    return packageTypes

def loadOrder(orderNo):
    order = pd.read_csv(f"data/Zamowienia/Zamowienie{orderNo}.csv")
    order.rename(columns={'Ilość': 'count'}, inplace=True)
    return order

types = loadPackageTypes()
print(types['x'].unique())
print(types['y'].unique())
print(types['z'].unique())

['600' '500' '800' '900' '400' '360' '560' '440' '300']
['400' '300' '600' '500' '240' '360' '320' '200']
['300' '240' '400' '360' '200' '160' '100']


In [89]:
# with open("file.sth", 'w') as wfile:
#     wfile.write("sth")

# packages = loadPackageTypes()
# print(packages)
def mnoz (tuples):
    wynik = 1
    for tuple in tuples:
        x,y,z = tuple
        wynik = wynik * z
    return wynik

def martyna(x):
    return [(400, [200, 500, 300]), (200, [200, 500, 300])]

sumaSum = 0
for i in range(1,21):
    df = loadOrder(i)
    df = df.join(types, on='ID Produktu', how='left', rsuffix='_types')
    size_df = df.drop(columns=['Nazwa Produktu_types', 'Nazwa Produktu','Lokacja', 'Waga (kg)', 'ID Produktu'])
    heights = df['z'].unique()
    suma = 0
    for height in heights:

        curr_df = size_df[size_df['z'] == height]
        curr_df = curr_df.groupby(['x', 'y']).sum()
        tuples = [(int(x), int(y), count) for (x, y), count in curr_df['count'].items()]
        layer_order = martyna(tuples)


        # tuples = [(row['x'], row['y'], row["count"]) for index, row in curr_df.iterrows()]
        # tuples = [row for index, row in curr_df.iterrows()]
        suma = suma + 1
        print(tuples, height)
    print(suma)
    sumaSum += suma
print(sumaSum)

# [
#     [
#         (600,400), (400,600)
#     ]
# ]
def fillRow(palette, packages, package_order,x,z):
    cursor = (x,0,z)
    for ID in package_order:
        package = packages.loc[ID]
        if cursor[0] + package['y'] <= palette.width:
            palette.addPackage(package, cursor)
            cursor = (cursor[0] + package['y'], cursor[1], cursor[2])

def fillLayer(palette, packages, package_orders, package_widths, z):
    cursor = (0,0,z)
    for i, package_order in enumerate(package_orders):
        fillRow(palette, packages, package_order, cursor[0], cursor[2])
        cursor = (package_widths[i] + cursor[0], cursor[1], cursor[2])

def package_order(raw_row, packages):



[(600, 400, 30)] 300
[(500, 300, 44), (560, 360, 22)] 240
[(360, 200, 19), (360, 240, 48)] 160
[(400, 300, 6), (440, 320, 23)] 200
4
[(560, 400, 27)] 300
[(440, 320, 55)] 200
[(360, 200, 23), (360, 240, 98)] 160
[(500, 300, 36), (560, 360, 14)] 240
4
[(360, 200, 23), (360, 240, 38)] 160
[(400, 300, 15), (440, 320, 5)] 200
[(500, 300, 12), (560, 360, 55)] 240
[(560, 400, 35)] 300
4
[(400, 300, 33), (440, 320, 61)] 200
[(360, 200, 7), (360, 240, 9)] 160
[(500, 300, 16), (560, 360, 14)] 240
[(560, 400, 14)] 300
[(600, 400, 10)] 360
5
[(500, 300, 6), (560, 360, 63)] 240
[(360, 200, 14), (360, 240, 15)] 160
[(600, 400, 1)] 360
[(500, 400, 3)] 300
[(440, 320, 8)] 200
5
[(600, 400, 9), (900, 500, 3)] 360
[(360, 240, 65)] 160
[(400, 300, 25), (440, 320, 33)] 200
[(600, 400, 23)] 300
[(560, 360, 11)] 240
5
[(560, 400, 51)] 300
[(400, 300, 4), (440, 320, 22)] 200
[(360, 200, 8), (360, 240, 48)] 160
[(560, 360, 22)] 240
4
[(360, 240, 53)] 160
[(560, 400, 13), (600, 400, 70)] 300
[(400, 300, 10), 

In [34]:
df = df.groupby('z')[['x', 'y']].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In [40]:
df.columns

Index(['ID Produktu', 'Lokacja', 'Ilość', 'Waga (kg)', 'x', 'y', 'z'], dtype='object')